In [26]:
from google.colab import drive
drive.mount('/content/drive')
% cd '/content/drive/My Drive/785_proj/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/785_proj


In [27]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [0]:
# !pip install import-ipynb
from torchvision import datasets
import torchvision.transforms as transforms
import import_ipynb


num_workers = 0
batch_size = 32
transform = transforms.Compose(
    [transforms.Resize((64,64)),
     transforms.ToTensor(),
     ])

train_data = datasets.ImageFolder(root = 'crop_part1', transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [29]:
from GeneratorUnet import * 
from D_G import *
from torchvision import models
import torch

G = UNet(3,3).to(device)
mobilenet = models.mobilenet_v2(pretrained=True)
mobilenet.classifier = nn.Sequential(
    nn.Linear(1280, 2),
    nn.Softmax(),
)
D_F = mobilenet.to(device)
D_G = torch.load('GD.pt').to(device)
# set gender discriminator to be fixed
for param in D_G.parameters():
    param.requires_grad = False

initilized!


In [0]:
"""
define loss
"""
def l2_loss_G(out, input):
    criterion = nn.MSELoss()
    loss = criterion(out, input)
    return loss

def gender_D_loss(out, label):
    batch_size = out.size(0)
    if label == 0:
        labels = torch.zeros(batch_size).to(device)
    if label == 1:
        labels = torch.ones(batch_size).to(device)
    criterion = nn.CrossEntropyLoss()
    return criterion(out, labels.long())

def true_D_loss(out):
    batch_size = out.size(0)
    labels = torch.ones(batch_size).to(device)
    criterion = nn.CrossEntropyLoss()
    return criterion(out, labels.long())

def fake_D_loss(out):
    batch_size = out.size(0)
    labels = torch.zeros(batch_size).to(device)
    criterion = nn.CrossEntropyLoss()
    return criterion(out, labels.long())

In [0]:
import torch.optim as optim

lr = 0.0002
d_optimizer = optim.Adam(D_F.parameters(), lr)
g_optimizer = optim.Adam(G.parameters(), lr)

In [0]:
# training
# training hyperparams
num_epochs = 100

# keep track of loss and generated, "fake" samples
samples = []
losses = []

print_every = 10

# train the network
D_F.train()
D_G.train()
G.train()
for epoch in range(num_epochs):
    
    for batch_i, (real_images, genders) in enumerate(train_loader):
                
        batch_size = real_images.size(0)
        real_images, genders = real_images.to(device), genders.to(device)

        # ============================================
        #            TRAIN THE DISCRIMINATOR
        # ============================================
        
        d_optimizer.zero_grad()
        
        # 1. Train with real images

        # Compute the discriminator losses on real images 
        D_real = D_F(real_images)
        d_real_loss = true_D_loss(D_real)
        
        # 2. Train with fake images
        
        # Generate fake female images
        fake_images_f = G(real_images, 0)
        fake_images_m = G(real_images, 1)

        # Compute the discriminator losses on fake images        
        D_fake_f = D_F(fake_images_f)
        d_fake_f_loss = fake_D_loss(D_fake_f)
        
        D_fake_m = D_F(fake_images_m)
        d_fake_m_loss = fake_D_loss(D_fake_m)
        # add up loss and perform backprop
        d_loss = d_real_loss + d_fake_f_loss + d_fake_m_loss
        d_loss.backward()
        d_optimizer.step()

        # =========================================
        #            TRAIN THE GENERATOR
        # =========================================
        g_optimizer.zero_grad()
        
        # 1. Train with l2loss
        # Generate fake female images
        fake_images_f = G(real_images, 0)
        fake_images_m = G(real_images, 1)
        l2loss_f = l2_loss_G(fake_images_f, real_images)
        l2loss_m = l2_loss_G(fake_images_m, real_images)

        # using flipped labels!

        # 2. Train with gender discriminator
        D_G_fake_f = D_G(fake_images_f)
        D_G_fake_m = D_G(fake_images_m)

        gender_loss_f = gender_D_loss(D_G_fake_f, 0)
        gender_loss_m = gender_D_loss(D_G_fake_m, 1)
        
        # 3. Train with face discriminator
        D_F_fake_f = D_F(fake_images_f)
        D_F_fake_m = D_F(fake_images_m)

        face_loss_f =  true_D_loss(D_F_fake_f)
        face_loss_m =  true_D_loss(D_F_fake_m)

        # 4.consistency loss
        fake_cycle = G(real_images, genders)
        c_loss = l2_loss_G(fake_cycle, real_images)

        # sum all the loss
        g_loss = l2loss_f + l2loss_m + gender_loss_f + gender_loss_m + face_loss_f + face_loss_m + c_loss
        # perform backprop
        g_loss.backward()
        g_optimizer.step()

        # Print some loss stats
        if batch_i % print_every == 0:
            # print discriminator and generator loss
            print('Epoch [{:5d}/{:5d}] | d_loss: {:6.4f} | g_loss: {:6.4f}'.format(
                    epoch+1, num_epochs, d_loss.item(), g_loss.item()))
            torch.save(G, 'G.pt')
            torch.save(D_F, 'FD.pt')
            
        ## AFTER EACH EPOCH##
        # append discriminator loss and generator loss
        losses.append((d_loss.item(), g_loss.item()))
        
        # # generate and save sample, fake images
        # G.eval() # eval mode for generating samples
        # samples_z = G(fixed_z)
        # samples.append(samples_z)
        # G.train() # back to train mode


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [    1/  100] | d_loss: 2.1923 | g_loss: 5.4506


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type UNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DoubleConv. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/d

Epoch [    1/  100] | d_loss: 1.9183 | g_loss: 5.4309
Epoch [    1/  100] | d_loss: 1.9109 | g_loss: 5.0453
Epoch [    1/  100] | d_loss: 1.8907 | g_loss: 4.6494
Epoch [    1/  100] | d_loss: 1.8560 | g_loss: 4.4814
Epoch [    1/  100] | d_loss: 1.7968 | g_loss: 4.3389


In [0]:
# visualization
import numpy as np
import matplotlib.pyplot as plt

#1 
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.to(device)
#2
G.eval()
fake_images_f = G(images, 0)
fake_images_m = G(images, 1)
img = images[8].cpu()
# plt.imshow(img.permute(1, 2, 0))
img2 = fake_images_f[8].detach().cpu()
plt.imshow(img2.permute(1, 2, 0))